#                The Influence of the Weather on Travel Behaviour in the Netherlands
### Project Group 04

*Members and Student Numbers:*

Hsuan-An Chu: 5914647\
Jarrik Overbosch: 6105734\
Mats Poppe: 5883245\
Cian Rippen: 5054141\
Sam Terstappen: 6078720



## 1. Introduction

## 2. Research Objective
(Maybe use of SMART tool?)


## What is the influence of the weather on travel behaviour in The Netherlands?
* What is the correlation between amount of percipitation and amount of transport passengers, based on OV chipcard checkins? 
* What is the effect of weather on road traffic?  
* What is the correlation between the weather and congestion?
* Is there a difference in behaviour between cities and rural areas?
* Where do the tipping points of behaviour lie? (e.g.amount of rain that starts people to change their behaviour)

(These maybe need some adjustments in the way they are formulated)

## 3. Data Pipeline
(Explain what data we used and why)


## 4. Results
(Answering the Sub Questions)


##### 1. 

##### 2.

##### 3. 

##### 4. 

##### 5. 

## 5. Conclusion 
(Answering the Central Research Question)

## 6. Discussion 
(Limitations and directions for further research)

## 7. Contribution Statement